<center><h1>Escenarios experimentales Internetworking</h1></center>

Presentado por: Rodney Christopher Armas Vacas
Fecha: 23/02/2023

# Importación de librerias necesarias

In [1]:
#This repo is to share the code shared during the TechGig webinar. 
#We have used a dataset taken from Canadian Institute of CyberSecurity. 
#We cannot share the data here; please contact a.habibi.l@unb.ca to obtain the dataset.
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Cargar el Dataset

In [2]:
datapath = 'https://raw.githubusercontent.com/hfcatucuamba/proyectoInternetworking/main/Escenario2.csv'
dataframe = pd.read_csv(datapath,low_memory=False)

In [3]:
dataframe.head()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,57188,82.161.239.177,110,6,7248168,2.112603e+04,29.110804,34515.085714,273869.262513,...,7,0,0,0,0,0,0,0,0,AUDIO
1,10.0.2.15,57188,82.161.239.177,110,6,5157723,1.052790e+03,3.683796,286540.166667,878838.525555,...,451,0,0,0,0,0,0,0,0,AUDIO
2,10.0.2.15,57188,82.161.239.177,110,6,41,1.324390e+07,48780.487805,41.000000,0.000000,...,0,0,0,0,0,0,0,0,0,AUDIO
3,10.0.2.15,57188,82.161.239.177,110,6,9543565,6.373855e+04,83.930900,11929.456250,90072.192523,...,5,0,0,0,0,0,0,0,0,AUDIO
4,10.0.2.15,57188,82.161.239.177,110,6,9987537,1.996166e+05,245.806348,4069.900978,20365.733229,...,5,0,0,0,0,0,0,0,0,AUDIO


# Normalizar los datos

In [4]:
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df

In [5]:
print (dataframe.shape)
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()
print(dataframe.describe())
print (list(dataframe))

(8044, 29)
        Source Port   Destination Port   Protocol   Flow Duration  \
count   8044.000000        8044.000000     8044.0    8.044000e+03   
mean   30505.505719       16767.761313        6.0    8.412146e+06   
std    22904.243121       22545.613741        0.0    3.062480e+06   
min      110.000000         110.000000        6.0    2.200000e+01   
25%      443.000000         443.000000        6.0    8.906492e+06   
50%    38894.000000         443.000000        6.0    9.965426e+06   
75%    51024.000000       41994.000000        6.0    9.989334e+06   
max    60740.000000       59708.000000        6.0    1.000000e+07   

        Flow Bytes/s   Flow Packets/s   Flow IAT Mean   Flow IAT Std  \
count   8.044000e+03      8044.000000    8.044000e+03   8.044000e+03   
mean    2.707062e+05       706.836422    1.017038e+05   2.436356e+05   
std     1.420024e+06      4391.154798    3.139844e+05   6.377622e+05   
min     0.000000e+00         0.400114    2.200000e+01   0.000000e+00   
25%    

In [6]:
keys = dataframe.keys()
# Feature Selection, Dropping Source IP, Source Port, Destination IP and Destination Port as it 
# specific to each organisation network and generic model should not contain them. 
data_to_process = dataframe[keys[4:len(keys) - 1]].copy()
#data_to_process = dataframe[[' Source Port',' Destination Port', ' Flow Duration', ' Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean','Fwd IAT Mean','Bwd IAT Mean','Active Mean','Idle Mean','label']].copy()
# do a data normalization
x_normalised = dfNormalize(data_to_process)
print(x_normalised.describe())

        Protocol   Flow Duration   Flow Bytes/s   Flow Packets/s  \
count     8044.0     8044.000000    8044.000000      8044.000000   
mean         0.0        0.841214       0.009988         0.007771   
std          0.0        0.306249       0.052394         0.048303   
min          0.0        0.000000       0.000000         0.000000   
25%          0.0        0.890649       0.000524         0.000332   
50%          0.0        0.996543       0.001092         0.000849   
75%          0.0        0.998933       0.006226         0.002472   
max          0.0        1.000000       1.000000         1.000000   

        Flow IAT Mean   Flow IAT Std   Flow IAT Max   Flow IAT Min  \
count     8044.000000    8044.000000    8044.000000    8044.000000   
mean         0.030513       0.042212       0.121525       0.017843   
std          0.094223       0.110499       0.202746       0.013580   
min          0.000000       0.000000       0.000000       0.000000   
25%          0.001327       0.002402 

In [7]:
#Preprocesa los datos dividiendo las características (features) y etiquetas (labels) y convirtiendo las etiquetas a valores numéricos utilizando la clase LabelEncoder de Scikit-learn
y = dataframe['label']

le = LabelEncoder()
y = le.fit_transform(y)

# Obtener los datos para entrenar y de prueba

In [8]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x_normalised,
                                            y, test_size=0.3, random_state=42)

# Creación modelo¶

In [10]:
# Crear el modelo de DNN
# Function to create model, required for KerasClassifier
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Crear modelo DNN
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

# Compilar modelo
model.compile(optimizer="adam",
              loss="binary_crossentropy", 
              metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1600      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,713
Trainable params: 3,713
Non-trainable params: 0
_________________________________________________________________


In [11]:
batch_size = 32
steps_per_epoch = (X_train.shape[0]//batch_size)
validation_steps = (x_normalised.shape[0] // batch_size)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

In [12]:
%%time

model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
176/176 [==============================] - 2s 4ms/step - loss: -35.4309 - accuracy: 0.1885 - val_loss: -47.0427 - val_accuracy: 0.1976
Epoch 2/10
176/176 [==============================] - 0s 2ms/step - loss: -46.5467 - accuracy: 0.2002 - val_loss: -47.0427 - val_accuracy: 0.1976
Epoch 3/10
176/176 [==============================] - 0s 2ms/step - loss: -46.5467 - accuracy: 0.2002 - val_loss: -47.0427 - val_accuracy: 0.1976
Epoch 4/10
176/176 [==============================] - 0s 2ms/step - loss: -46.5467 - accuracy: 0.2002 - val_loss: -47.0427 - val_accuracy: 0.1976
Epoch 5/10
176/176 [==============================] - 0s 2ms/step - loss: -46.5468 - accuracy: 0.2002 - val_loss: -47.0427 - val_accuracy: 0.1976
Epoch 6/10
176/176 [==============================] - 0s 2ms/step - loss: -46.5467 - accuracy: 0.2002 - val_loss: -47.0427 - val_accuracy: 0.1976
Epoch 7/10
176/176 [==============================] - 0s 2ms/step - loss: -46.5467 - accuracy: 0.2002 - val_loss: -47.0427 -

In [13]:
loss, accuracy = model.evaluate(X_test,y_test,batch_size=batch_size)
# Evaluar el modelo
print('test_loss: %.2f' % (loss * 100.0))  
print('test_accuracy: %.2f' % (accuracy * 100.0))

76/76 [==============================] - 0s 1ms/step - loss: -47.0427 - accuracy: 0.1976
test_loss: -4704.27
test_accuracy: 19.76


In [18]:
# Importar métricas de sklearn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Hacer predicciones sobre los datos de prueba
y_pred = model.predict(X_test)

# Convertir las probabilidades en etiquetas binarias (0 o 1) usando un umbral de 0.5
y_pred_binary = (y_pred > 0.5).astype(int)

# Calcular métricas
acc = accuracy_score(y_test, y_pred_binary)*100
f1 = f1_score(y_test, y_pred_binary, average='weighted')*100
prec = precision_score(y_test, y_pred_binary, average='weighted')*100
rec = recall_score(y_test, y_pred_binary, average='weighted')*100

# Imprimir métricas
print("Accuracy:", acc)
print("F1 Score:", f1)
print("Precision:", prec)
print("Recall:", rec)

76/76 [==============================] - 0s 2ms/step
Accuracy: 19.75973487986744
F1 Score: 6.520507462951761
Precision: 3.904471225226499
Recall: 19.75973487986744


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
